# Discogs Functions

In [5]:
## Basic stuff
%load_ext autoreload
%autoreload
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
display(HTML("""<style>div.output_area{max-height:10000px;overflow:scroll;}</style>"""))

## Python Version
import sys
print("Python: {0}".format(sys.version))

from ioUtils import saveJoblib, loadJoblib, saveFile, getFile
import urllib
from urllib.parse import quote

from discogsBase import discogs
from discogsUtils import discogsUtils
from collection import collections
from artist import artist
from timeUtils import clock, elapsed
from fsUtils import moveFile, setFile, setDir, setSubDir, isFile, isDir, mkDir
from fileUtils import getFileBasics, getBasename
from artists import artists
from albums import albums
from time import sleep

import datetime as dt
start = dt.datetime.now()
print("Notebook Last Run Initiated: "+str(start))

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Python: 3.7.3 (default, Mar 27 2019, 16:54:48) 
[Clang 4.0.1 (tags/RELEASE_401/final)]
Notebook Last Run Initiated: 2019-08-08 00:27:02.787445


# Discogs Base Class

In [6]:
%load_ext autoreload
%autoreload
disc = discogs()
arts = artists(disc)
dutils = discogsUtils()
col = collections(disc)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Saved Discog Directory /Volumes/Music/Discog is Available
Warning! Local Discog Directory /Users/tgadfort/Documents/music/Discog is not Available
/Volumes/Music/Discog/base exists
/Volumes/Music/Discog/collections exists
/Volumes/Music/Discog/artists exists
/Volumes/Music/Discog/albums exists
/Volumes/Music/Discog/base-db exists
/Volumes/Music/Discog/collections-db exists
/Volumes/Music/Discog/artists-db exists
/Volumes/Music/Discog/albums-db exists
/Volumes/Music/Discog/artists-extra exists
/Volumes/Music/Discog/search exists
/Volumes/Music/Discog/search-artists exists
/Volumes/Music/Discog/special exists
/Volumes/Music/Discog/artist-special exists
/Volumes/Music/Discog/db exists
Found 100 artist DB files and that is equal to the max mod value


# Artist Section

In [ ]:
db = disc.getArtistIDCoreAlbumNames()

In [ ]:
artistIDs = disc.getArtistNameToIDsData()

In [120]:
IDToName  = disc.getArtistIDToNameData()
NameToIDs = disc.getArtistNameToIDsData()

Loading data from /Volumes/Music/Discog/db/ArtistIDToName.p
  --> This file is 7.6MB.
Loading /Volumes/Music/Discog/db/ArtistIDToName.p
Loading data from /Volumes/Music/Discog/db/ArtistVariationNameToIDs.p
  --> This file is 17.8MB.
Loading /Volumes/Music/Discog/db/ArtistVariationNameToIDs.p
Loading data from /Volumes/Music/Discog/db/ArtistIDCoreAlbumNames.p
  --> This file is 42.8MB.
Loading /Volumes/Music/Discog/db/ArtistIDCoreAlbumNames.p


In [146]:
ArtistAlbums = disc.getArtistIDCoreAlbumIDs()
AlbumIDToName = disc.getAlbumIDToNameData()

Loading data from /Volumes/Music/Discog/db/ArtistIDCoreAlbumIDs.p
  --> This file is 22.5MB.
Loading /Volumes/Music/Discog/db/ArtistIDCoreAlbumIDs.p
Loading data from /Volumes/Music/Discog/db/AlbumIDToName.p
  --> This file is 81.8MB.
Loading /Volumes/Music/Discog/db/AlbumIDToName.p


ValueError: read of closed file

In [125]:
from string import ascii_uppercase, digits, punctuation
vals = "{0}{1}{2}".format(ascii_uppercase, digits, punctuation)
AsciiToName = {val: [x for x in NameToIDs.keys() if (x.startswith(val) or x.startswith(val.lower()))] for val in vals}

In [166]:
from discogsBase import discogs
from string import ascii_uppercase, digits, punctuation
from searchUtils import findNearest

class discdb:
    def __init__(self, disc):
        self.dutils    = discogsUtils()
        self.disc      = disc
        
        ## DBs
        self.IDToName  = None
        self.NameToIDs = None
        self.AsciiToName = None
        self.ArtistAlbums = None
        self.AlbumIDToName = None
        
        self.artistRetval = None
        self.minCutoff    = 0.85
        self.minN         = 1
        
        
    def loadAlbumDBs(self, ArtistAlbums=None, AlbumIDToName=None):
        start, cmt = clock("Loading Album DBs")
        if ArtistAlbums is None:
            self.ArtistAlbums = disc.getArtistIDCoreAlbumNames()
        else:
            self.ArtistAlbums = ArtistAlbums
            
        if AlbumIDToName is None:
            self.AlbumIDToName  = disc.getAlbumIDToNameData()
        else:
            self.AlbumIDToName  = AlbumIDToName
        elapsed(start, cmt)
        
        
    def loadArtistDBs(self, IDToName=None, NameToIDs=None, AsciiToName=None):
        start, cmt = clock("Loading Artist DBs")
        
        if IDToName is None:
            self.IDToName  = disc.getArtistIDToNameData()
        else:
            self.IDToName  = IDToName
            
        if NameToIDs is None:
            self.NameToIDs = disc.getArtistNameToIDsData()
        else:
            self.NameToIDs = NameToIDs
            
        if AsciiToName is None:
            vals = "{0}{1}{2}".format(ascii_uppercase, digits, punctuation)
            self.AsciiToName = {val: [x for x in n_items if (x.startswith(val) or x.startswith(val.lower()))] for val in vals}
        else:
            self.AsciiToName = AsciiToName
            
        elapsed(start, cmt)

        
    def findArtist(self, name, num=None, cutoff=None):
        print("Searching For {0}".format(name))
        try:
            charval = str(name)[0]
        except:
            raise ValueError("Artist must be a string Type=[{0}]".format(type(name)))
            
        if num is None:
            num = self.minN
            
        if cutoff is None:
            cutoff = self.minCutoff

        chardata = self.AsciiToName.get(charval)
        if chardata is None:
            raise ValueError("There are no entries for character [{0}]".format(charval))
        retval = findNearest(name, chardata, num=num, cutoff=cutoff)
        print("Results: {0}".format(retval))
        self.artistRetval = retval
        
        return retval
    
    
    def findArtistAlbums(self, artist=None):
        if artist is None:
            artist = self.artistRetval
        if artist is None:
            print("Artist is None!")
            return
        
        if isinstance(artist, str):
            artists = [artist]
        else:
            artists = artist
            
        artistIDs = []
        for artist in artists:
            IDs = self.NameToIDs.get(artist)
            if isinstance(IDs,list):
                retval = [ID for ID in IDs if ID is not None]
                artistIDs += retval                
        artistIDs = list(set(artistIDs))

        for artistID in artistIDs:
            artist = self.IDToName[artistID]
            print("Artist [{0}]  ---->  [{1}]".format(artist, artistID))
            albumIDs = ArtistAlbums.get(artistID)
            print("  ID [{0}]  ---->  {1} albums".format(artistID, len(albumIDs)))
            for albumID in albumIDs:
                albumNames = self.AlbumIDToName.get(albumID)
                if isinstance(albumNames, dict):
                    albumNames = list(albumNames.keys())
                print('\t{0: <12}\t{1}'.format(albumID, albumNames))
            
        

In [167]:
ddb = discdb(disc)
ddb.loadArtistDBs(IDToName=IDToName, NameToIDs=NameToIDs, AsciiToName=AsciiToName)
ddb.loadAlbumDBs(ArtistAlbums=ArtistAlbums, AlbumIDToName=AlbumIDToName)
ddb.findArtist("Prince", num=1)
ddb.findArtistAlbums()

Current Time is Wed Aug 07, 2019 07:58:22 for Loading Artist DBs
Current Time is Wed Aug 07, 2019 07:58:22 for Done with Loading Artist DBs
Process [{0}] took 277 microseconds.
Current Time is Wed Aug 07, 2019 07:58:22 for Loading Album DBs
Current Time is Wed Aug 07, 2019 07:58:22 for Done with Loading Album DBs
Process [{0}] took 61 microseconds.
Searching For Prince
Results: ['Prince']
Artist [Alan Price]  ---->  [283847]
  ID [283847]  ---->  91 albums
	3001460     	['The Story Of Alan Price']
	269974      	['Between Today And Yesterday']
	6042763     	['Travellinâ\x80\x99 Man']
	458012      	['This Is Your Lucky Day']
	2788320     	['To Ramona / Not Born To Follow']
	672500      	["Don't Hit Me When I'm Down", 'Wait Until The Morning (Estereo: Remixed)']
	647226      	['Rosetta / Bonnie And Clyde']
	418764      	['O Lucky Man!']
	3224803     	['Georgie Fame With Alan Price']
	334017      	['Untitled', 'Just For You']
	388628      	['The Trimdon Grange Explosion / Falling In Love A

	5192412     	['The Work, Part I / U Make My Sun Shine']
	109246      	['If I Was Your Girlfriend']
	766491      	['The Story So Far...']
	2148149     	['The Hits']
	7899105     	['When Will We B Paid?']
	5115454     	['Paisley Park / Frankie']
	194670      	['Wake The World', 'Still Waiting']
	3119078     	['Automatic']
	506521      	['Da Underground King', 'Hot Thing']
	6835885     	['Cause And Effect']
	109430      	['The Rainbow Children']
	7503386     	['Head To Head']
	8985381     	["U're Gonna C Me "]
	394236      	['Dance 4 Me']
	1571587     	["Let's Go Crazy"]
	109227      	['Black Sweat / Beautiful, Loved and Blessed']
	1565224     	['Aviation', 'Originals']
	1009072     	['Gett Off Remix EP']
	97252       	['Gett Off']
	55667       	['1999']
	55672       	["Searchin'", 'Fury']
Artist [Viv Prince]  ---->  [941914]
  ID [941914]  ---->  1 albums
	6947195     	['Light Of The Charge Brigade']
Artist [Prince Jazzbo]  ---->  [94504]
  ID [94504]  ---->  159 albums
	5756251     	['

In [ ]:
%load_ext autoreload
%autoreload
disc   = discogs()
arts   = artists(disc)
albs   = albums(disc)
dutils = discogsUtils()
col    = collections(disc)

for modVal in list(range(99, 50, -1)):
    albs.downloadAlbumModValData(modVal, maxAlbums=1)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Saved Discog Directory /Volumes/Music/Discog is Available
Warning! Local Discog Directory /Users/tgadfort/Documents/music/Discog is not Available
/Volumes/Music/Discog/base exists
/Volumes/Music/Discog/collections exists
/Volumes/Music/Discog/artists exists
/Volumes/Music/Discog/albums exists
/Volumes/Music/Discog/base-db exists
/Volumes/Music/Discog/collections-db exists
/Volumes/Music/Discog/artists-db exists
/Volumes/Music/Discog/albums-db exists
/Volumes/Music/Discog/artists-extra exists
/Volumes/Music/Discog/search exists
/Volumes/Music/Discog/search-artists exists
/Volumes/Music/Discog/special exists
/Volumes/Music/Discog/artist-special exists
/Volumes/Music/Discog/db exists
Found 100 artist DB files and that is equal to the max mod value
Loading /Volumes/Music/Discog/artists-db/99-DB.p... Found 4404 Artists
1 / 4404 	: 1002799 	 Fred Pellerin
	10 entries of type Albums
  Saving /Volumes/Music

45 / 4404 	: 1049899 	 SXL (2)
	2 entries of type Albums
  Saving /Volumes/Music/Discog/albums/99/1049899/1086076.p
  --> This file is 16.8kB.
46 / 4404 	: 10499 	 LTJ X-Perience
	5 entries of type Albums
  Saving /Volumes/Music/Discog/albums/99/10499/261746.p
  --> This file is 19.6kB.
47 / 4404 	: 1055299 	 G.L.D. Project
48 / 4404 	: 1055499 	 Sophie Watillon
	2 entries of type Albums
  Saving /Volumes/Music/Discog/albums/79/864379/6189885.p
  --> This file is 18.8kB.
  Saving /Volumes/Music/Discog/albums/99/1055499/6189885.p
  --> This file is 18.8kB.
49 / 4404 	: 1055999 	 Funkymic
	2 entries of type Albums
  Saving /Volumes/Music/Discog/albums/99/1055999/10374615.p
  --> This file is 15.6kB.
  Saving /Volumes/Music/Discog/albums/0/1056000/10374615.p
  --> This file is 15.6kB.
50 / 4404 	: 1057499 	 Cock And Swan
	5 entries of type Albums
  Saving /Volumes/Music/Discog/albums/99/1057499/1258944.p
  --> This file is 15.4kB.
51 / 4404 	: 1057899 	 New Hope
	9 entries of type Albums


  Saving /Volumes/Music/Discog/albums/99/110899/1428349.p
  --> This file is 17.8kB.
93 / 4404 	: 11099 	 The Goodfellas (2)
94 / 4404 	: 1112499 	 Ide (4)
	12 entries of type Albums
  Saving /Volumes/Music/Discog/albums/99/1112499/1748586.p
  --> This file is 19.2kB.
95 / 4404 	: 1112699 	 Boomerang (7)
96 / 4404 	: 1112999 	 The Partycrushers
97 / 4404 	: 1116099 	 Ansambl Renesans
	7 entries of type Albums
  Saving /Volumes/Music/Discog/albums/99/1116099/3096826.p
  --> This file is 18.8kB.
98 / 4404 	: 1116199 	 Opus Avantra
	6 entries of type Albums
  Saving /Volumes/Music/Discog/albums/99/1116199/177328.p
  --> This file is 19.0kB.
99 / 4404 	: 111699 	 FM (3)
	9 entries of type Albums
  Saving /Volumes/Music/Discog/albums/99/111699/168985.p
  --> This file is 21.5kB.
100 / 4404 	: 1117299 	 Kullervo Kojo
	3 entries of type Albums
  Saving /Volumes/Music/Discog/albums/49/4820549/7973952.p
  --> This file is 16.0kB.
  Saving /Volumes/Music/Discog/albums/4/936804/7973952.p
  --> Th

  Saving /Volumes/Music/Discog/albums/99/1166299/2655608.p
  --> This file is 18.8kB.
146 / 4404 	: 1167399 	 Greg Copeland
	2 entries of type Albums
  Saving /Volumes/Music/Discog/albums/99/1167399/115870.p
  --> This file is 18.4kB.
147 / 4404 	: 11699 	 Shane
148 / 4404 	: 1170399 	 Twentieth Century Classics Ensemble
	4 entries of type Albums
  Saving /Volumes/Music/Discog/albums/83/465983/3328603.p
  --> This file is 20.2kB.
  Saving /Volumes/Music/Discog/albums/99/1170399/3328603.p
  --> This file is 20.2kB.
  Saving /Volumes/Music/Discog/albums/87/2567287/3328603.p
  --> This file is 20.2kB.
  Saving /Volumes/Music/Discog/albums/86/1341786/3328603.p
  --> This file is 20.2kB.
  Saving /Volumes/Music/Discog/albums/92/333892/3328603.p
  --> This file is 20.2kB.
149 / 4404 	: 1174799 	 Von Groove
	7 entries of type Albums
  Saving /Volumes/Music/Discog/albums/99/1174799/750579.p
  --> This file is 19.2kB.
150 / 4404 	: 1176299 	 Roy England
	2 entries of type Albums
  Saving /Volum

192 / 4404 	: 1240099 	 The Yes-Men
	1 entries of type Albums
193 / 4404 	: 124199 	 Click Productions
194 / 4404 	: 1245499 	 Sophie Hunger
	6 entries of type Albums
  Saving /Volumes/Music/Discog/albums/99/1245499/811228.p
  --> This file is 17.4kB.
195 / 4404 	: 1249599 	 Gabriel G
	1 entries of type Albums
196 / 4404 	: 1250799 	 Christian Rich
	1 entries of type Albums
  Saving /Volumes/Music/Discog/albums/99/1250799/873276.p
  --> This file is 17.3kB.
197 / 4404 	: 125299 	 J.E.N.
198 / 4404 	: 1253199 	 Bobby Bandiera
	2 entries of type Albums
  Saving /Volumes/Music/Discog/albums/99/1253199/8895482.p
  --> This file is 17.4kB.
199 / 4404 	: 1254299 	 Serum 114
	6 entries of type Albums
  Saving /Volumes/Music/Discog/albums/99/1254299/696022.p
  --> This file is 17.1kB.
200 / 4404 	: 1259399 	 Joel "Twinkles" Smith
201 / 4404 	: 1260599 	 Koufar
	9 entries of type Albums
  Saving /Volumes/Music/Discog/albums/99/1260599/1727344.p
  --> This file is 17.0kB.
202 / 4404 	: 126099 	 

  Saving /Volumes/Music/Discog/albums/96/8796/12459350.p
  --> This file is 19.4kB.
249 / 4404 	: 1325299 	 Krister Malm
	1 entries of type Albums
250 / 4404 	: 1326399 	 Doran Usher
251 / 4404 	: 1326499 	 Steve Sasberg Ansah
252 / 4404 	: 132799 	 Mo-Dettes
	1 entries of type Albums
  Saving /Volumes/Music/Discog/albums/99/132799/85553.p
  --> This file is 20.7kB.
253 / 4404 	: 1329699 	 Thomas Streutgers
254 / 4404 	: 1334299 	 Fuguett Luz
255 / 4404 	: 133599 	 Jukka Poika & Kompostikopla
	1 entries of type Albums
256 / 4404 	: 133699 	 The Union (2)
	4 entries of type Albums
  Saving /Volumes/Music/Discog/albums/99/133699/753493.p
  --> This file is 18.3kB.
257 / 4404 	: 1338399 	 Sherrié Austin
	5 entries of type Albums
  Saving /Volumes/Music/Discog/albums/99/1338399/628808.p
  --> This file is 18.7kB.
258 / 4404 	: 1340899 	 John Sferra
259 / 4404 	: 1343099 	 Christiane Obydol
	1 entries of type Albums
260 / 4404 	: 1344199 	 Trash Talk
	5 entries of type Albums
  Saving /Volu

  Saving /Volumes/Music/Discog/albums/64/125764/2133930.p
  --> This file is 17.2kB.
307 / 4404 	: 1391599 	 Hollace M. Metzger
308 / 4404 	: 1391699 	 Soufiane Amrani
309 / 4404 	: 1392299 	 Blóðtrú
	5 entries of type Albums
  Saving /Volumes/Music/Discog/albums/99/1392299/438809.p
  --> This file is 17.0kB.
310 / 4404 	: 1393099 	 Adam (48)
311 / 4404 	: 1393299 	 Arm Of Roger
	1 entries of type Albums
  Saving /Volumes/Music/Discog/albums/99/1393299/897704.p
  --> This file is 17.9kB.
312 / 4404 	: 1394399 	 Rimantas MotiejÅ«nas
313 / 4404 	: 1394599 	 Maetake
314 / 4404 	: 1394799 	 Andrea Ughetto
	1 entries of type Albums
  Saving /Volumes/Music/Discog/albums/99/1394799/1651731.p
  --> This file is 16.4kB.
315 / 4404 	: 1395499 	 Wulff Raven
316 / 4404 	: 1395799 	 Hundred Million Martians
	4 entries of type Albums
  Saving /Volumes/Music/Discog/albums/99/1395799/2843383.p
  --> This file is 16.5kB.
317 / 4404 	: 1396199 	 I-Massive
318 / 4404 	: 1396499 	 Michael Minsky
	7 entrie

367 / 4404 	: 1462399 	 J. Van Kerckhoven
368 / 4404 	: 1463799 	 Christy Azuma
	2 entries of type Albums
  Saving /Volumes/Music/Discog/albums/99/1463799/851191.p
  --> This file is 17.8kB.
  Saving /Volumes/Music/Discog/albums/97/1463797/851191.p
  --> This file is 17.8kB.
369 / 4404 	: 1465499 	 Doris (9)
370 / 4404 	: 146599 	 Ikon (4)
	10 entries of type Albums
371 / 4404 	: 1466599 	 Vinzent Kutsche
372 / 4404 	: 1468599 	 Charlie McBurney
373 / 4404 	: 1470099 	 Luv For Luv
	1 entries of type Albums
374 / 4404 	: 1471099 	 Christian Naumann
375 / 4404 	: 1471799 	 Djole
376 / 4404 	: 1472399 	 Valis (5)
	4 entries of type Albums
  Saving /Volumes/Music/Discog/albums/99/1472399/551368.p
  --> This file is 18.1kB.
377 / 4404 	: 1473199 	 Sebastian Krechlok
378 / 4404 	: 1474899 	 Arianna Wilson
379 / 4404 	: 1476299 	 Vladimir Waskiw
380 / 4404 	: 147699 	 Clive Bell
	17 entries of type Albums
  Saving /Volumes/Music/Discog/albums/7/338607/2839705.p
  --> This file is 16.8kB.
  Sa

425 / 4404 	: 1512399 	 Maureen Nilsen
426 / 4404 	: 1515999 	 Slavebreed
	2 entries of type Albums
  Saving /Volumes/Music/Discog/albums/99/1515999/652301.p
  --> This file is 17.6kB.
427 / 4404 	: 1517699 	 Radim Smetana
428 / 4404 	: 1518499 	 Marisela
	13 entries of type Albums
  Saving /Volumes/Music/Discog/albums/99/1518499/1057972.p
  --> This file is 16.4kB.
429 / 4404 	: 1519099 	 Chalfont
	3 entries of type Albums
430 / 4404 	: 15199 	 Meagashira
431 / 4404 	: 1520899 	 George (29)
432 / 4404 	: 1522299 	 The Way (2)
	2 entries of type Albums
  Saving /Volumes/Music/Discog/albums/99/1522299/1449042.p
  --> This file is 17.3kB.
433 / 4404 	: 152299 	 The Glass Family
	3 entries of type Albums
  Saving /Volumes/Music/Discog/albums/99/152299/393606.p
  --> This file is 21.6kB.
434 / 4404 	: 1522999 	 Simmonds & Jones
	1 entries of type Albums
  Saving /Volumes/Music/Discog/albums/99/1522999/1507063.p
  --> This file is 17.1kB.
435 / 4404 	: 1523799 	 Tom Hull
436 / 4404 	: 15239

In [ ]:
disc.getA

In [ ]:
from searchUtils import findNearest
def findIDs
ddb

In [33]:
from itertools import islice
def takeN(n, iterable):
    "Return first n items of the iterable as a list"
    return list(islice(iterable, n))

n_items = takeN(500000, ddb.NameToIDs.keys())
n_items = ddb.NameToIDs.keys()

In [71]:
from string import punctuation
punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [79]:
from string import ascii_uppercase, digits, punctuation
vals = "{0}{1}{2}".format(ascii_uppercase, digits, punctuation)
vals

'ABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [80]:
retval = {val: [x for x in n_items if (x.startswith(val) or x.startswith(val.lower()))] for val in vals}

In [85]:
print(test)
for k,v in retval.items():
    print(k, findNearest(test, v, num=1, cutoff=0.8))

Poul 3 RudersX
A []
B []
C []
D []
E []
F []
G []
H []
I []
J []
K []
L []
M []
N []
O []
P ['Poul Ruders']
Q []
R []
S []
T []
U []
V []
W []
X []
Y []
Z []
0 []
1 []
2 []
3 []
4 []
5 []
6 []
7 []
8 []
9 []
! []
" []
# []
$ []
% []
& []
' []
( []
) []
* []
+ []
, []
- []
. []
/ []
: []
; []
< []
= []
> []
? []
@ []
[ []
\ []
] []
^ []
_ []
` []
{ []
| []
} []
~ []


In [74]:
for k,v in retval.items():
    print(k,len(v))

A 56789
B 54805
C 54169
D 69758
E 31362
F 28962
G 36485
H 29656
I 13396
J 72380
K 31234
L 42960
M 78213
N 22963
O 19445
P 42893
Q 1575
R 52819
S 70262
T 82300
U 4933
V 14074
W 24563
X 1764
Y 7705
Z 6093
0 155
1 892
2 867
3 598
4 435
5 267
6 168
7 217
8 195
9 137
! 37
" 3100
# 14
$ 73
% 1
& 19
' 717
( 464
) 2
* 28
+ 36
, 27
- 95
. 146
/ 28
: 51
; 0
< 9
= 6
> 16
? 37
@ 74
[ 170
\ 6
] 3
^ 3
_ 30
` 4
{ 7
| 8
} 2
~ 5


In [41]:
from searchUtils import findNearest
test = "Poul 3 RudersX"

In [76]:
start, cmt = clock("Test")
retval = []
ifn    = {0: 0.95, 1: 0.80}
for i,cutoff in ifn.items():
    retval = findNearest(test, retval[test[0]], num=1, cutoff=cutoff)
    if len(retval) > 0:
        break
elapsed(start, cmt)
print(retval)



start, cmt = clock("Test")
retval = []
ifn    = {0: 0.95}
for i,cutoff in ifn.items():
    retval = findNearest(test, retval[test[0]], num=1, cutoff=cutoff)
    if len(retval) > 0:
        break
elapsed(start, cmt)
print(retval)

Current Time is Mon Aug 05, 2019 19:42:25 for Test


TypeError: list indices must be integers or slices, not str

Current Time is Mon Aug 05, 2019 19:01:45 for Test
Current Time is Mon Aug 05, 2019 19:01:50 for Done with Test
Process [{0}] took 4 seconds.
['Poul Ruders']


In [16]:
for i,(k,v) in enumerate(ddb.IDToName.items()):
    print(i,k,v)
    if i > 50:
        break

0 1000 Dave Clarke
1 1000500 Club Pulse
2 100200 Dike
3 1002000 Larry Stokes
4 1002600 Gidd Sanchez
5 1002900 John Keding
6 1003900 Jason Pancho
7 1005400 Gapeseed
8 1007400 The Beauty Shop
9 1010100 The Salads
10 1013700 Mother Mother
11 1014400 Lucious Randolph
12 1016300 Poul Ruders
13 101700 Lanu
14 102100 Extrabreit
15 1027500 Luigi Barion
16 1028100 Keith Grant (2)
17 10300 Andre Harris
18 103000 C. Reider
19 1030700 William Schumann
20 1031700 Signal Path
21 103200 Shipping News
22 1034400 Gunnar Graps
23 1036400 The Helmet Kidz
24 1038800 Frank Petty Trio
25 1041000 dcee
26 1041100 Taro Bando
27 104200 The Jammers
28 1043700 Car-Man
29 1045400 Sol Hoopii
30 1046200 David PeÃ±a Dorantes
31 1046400 Ole Kock Hansen
32 1047400 Priscilla Ahn
33 1047700 Jay Perkins
34 1048600 Screaming Headless Torsos
35 105300 Schnitt Acht
36 1054600 Viv Akauldren
37 1057300 Rocky Athas
38 1057600 Joyce Kelly
39 10600 Victor Davies
40 1064300 Lilitu
41 1065000 Orchestre Philharmonique De Strasbourg


In [ ]:
for i,(k,v) in enumerate(ddb.NameToIDs.items()):
    print(i,k,v)
    if i > 50:
        break

In [ ]:
%load_ext autoreload
%autoreload
from artist import artist
from artists import artists
art  = artist()
arts = artists(disc)
if False:
    arts.parseArtistFiles(force=False)
if False:
    for modVal in range(0, 100):
        arts.parseArtistModValFiles(modVal)#art.searchDiscogForArtist("$uicideboy$")
arts.buildMetadata(force=True)



In [ ]:
dbArt = disc.getArtistIDToNameData()
dbCol = disc.getCollectionIDToRefData()
artistDBDir = disc.getArtistsDBDir()

In [ ]:
refs = [dbCol.get('1930579')]
for ref in refs:
    if ref is None:
        continue
    print("REF --> {0}".format(ref))
    href = ref.replace("https://www.discogs.com/artist/", "")
    artistID = dutils.getArtistID(ref)
    modValue = dutils.getDiscIDHashMod(discID=artistID, modval=disc.getMaxModVal())

    if False:
        dbname  = setFile(artistDBDir, "{0}-DB.p".format(modValue))     
        print("Loading {0}".format(dbname))
        dbdata  = getFile(dbname)

        try:
            del dbdata[artistID]
            print("Deleted {0}".format(artistID))
        except:
            print("Not there...")

        print("Saving {0}".format(dbname))
        saveFile(idata=dbdata, ifile=dbname)

    url = arts.getArtistURL(ref)
    savename = arts.getArtistSavename(artistID)
    arts.downloadArtistURL(url=url, savename=savename, parse=True, force=True, debug=True)
    sleep(1)

In [ ]:
arts.parseArtistModValFiles(modVal=89, force=True)

In [ ]:
errs=[]
for modVal in range(51,52):
    print("\n===============================================================")
    print("assertDBModValData({0})".format(modVal))
    if True:
        try:
            arts.assertDBModValData(modVal)
        except:
            errs.append(modVal)
    if False:
        try:
            arts.assertDBModValExtraData(modVal)
        except:
            errs.append(modVal)

In [ ]:
data = art.getData('/Volumes/Music/Discog/artists/22/2047522.p')

In [ ]:
errs

In [ ]:
arts.parseArtistModValFiles(modVal=51, force=True)

In [ ]:
for modVal in range(95, 100):
    print("ModVal = {0}".format(modVal))
    arts.assertDBModValData(modVal)
    
#arts.assertDBModValExtraData(modVal)

In [ ]:
dbArt = disc.getArtistIDToNameData()
dbCol = disc.getCollectionIDToRefData()
artistDBDir = disc.getArtistsDBDir()

In [ ]:
arts.rmIDFromDB('334885', '85')

In [ ]:
vals="""3,818 Vocal
2,446 Rock & Roll
2,014 Easy Listening
1,640 Romantic
1,200 Big Band
1,004 Swing
887 Classical
692 Opera
634 Country
582 Dixieland
560 Ballad
550 Bop
541 Folk
500 Soundtrack
477 Musical
462 Cool Jazz
447 Baroque
443 Rhythm & Blues
433 Modern
393 Rockabilly
363 Pop Rock
307 Doo Wop
274 Novelty
249 Theme
208 Calypso
203 Skiffle
200 Religious
180 Celtic
179 Ragtime
177 Hard Bop
174 Comedy
170 Spoken Word
167 Instrumental
133 Latin Jazz
127 Cha-Cha
127 Mambo
119 Gospel
117 Score
106 Chanson
99 Space-Age
96 Impressionist
94 Music Hall
91 Choral
91 Soul
90 Contemporary Jazz
88 Blues Rock
86 Post Bop
81 Light Music
79 Marches
78 Poetry
66 Operetta
65 Country Rock
64 Piano Blues
59 Neo-Classical
59 Parody
56 Neo-Romantic
55 Rumba
54 African
54 Tango
53 Honky Tonk
52 Afro-Cuban Jazz
52 Country Blues
52 Field Recording
52 Story
51 Soul-Jazz
50 Afro-Cuban
50 Pacific
43 Brass Band
43 Samba
40 Nursery Rhymes
39 Contemporary
39 Military
39 Renaissance
38 Dialogue
38 Flamenco
38 Schlager
33 Pipe & Drum
31 Bolero
25 Canzone Napoletana
25 Classic Rock
25 Radioplay
24 Boogie Woogie
24 Fado
24 Monolog
23 Technical
22 Bluegrass
21 Lounge
21 Mento
21 Oratorio
21 Surf
20 Special Effects
19 Delta Blues
19 Education
19 Gypsy Jazz
19 Polka
18 Cubano
18 Modal
17 Jump Blues
17 Louisiana Blues
15 Audiobook
15 Medieval
14 Chicago Blues
14 Merengue
13 Highlife
12 Beguine
12 Hillbilly
12 Soft Rock
11 Beat
11 Speech
10 Bossa Nova
10 Romani
9 Folk Rock
9 Interview
8 Piedmont Blues
8 Political
8 Quechua
8 Texas Blues
7 Avant-garde Jazz
7 Educational
7 Pasodoble
7 Salsa
6 East Coast Blues
6 Experimental
6 Harmonica Blues
6 Smooth Jazz
5 Andalusian Classical
5 Free Jazz
5 Gamelan
5 Hindustani
5 Sea Shanties
5 Sermon
5 Western Swing
4 Europop
4 Free Improvisation
4 Twelve-tone
4 Volksmusik
3 Abstract
3 Cajun
3 Funk
3 Hawaiian
3 Indian Classical
3 Musique Concrète
3 Steel Band
2 Acoustic
2 Afrobeat
2 Bachata
2 Baião
2 Batucada
2 Cape Jazz
2 Charanga
2 Contemporary R&B
2 Copla
2 Health-Fitness
2 Jazz-Funk
2 Laïkó
2 Mariachi
2 Post-Modern
2 Public Service Announcement
2 RnB/Swing
2 Zarzuela
1 Barbershop
1 Bollywood
1 Bossanova
1 Chinese Classical
1 Danzon
1 Electric Blues
1 Forró
1 Garage Rock
1 MPB
1 Maloya
1 Marcha Carnavalesca
1 Marimba
1 Modern Classical
1 Promotional
1 Public Broadcast
1 Rebetiko
1 Samba-Canção
1 Soca
1 Sound Poetry
1 Swamp Pop"""

In [ ]:
cnts   = [x.split()[0] for x in vals.split("\n")]
styles = [" ".join(x.split()[1:]) for x in vals.split("\n")]
d2000  = dict(zip(cnts, styles))
saveFile(idata=d2000, ifile="1950UKstyles.p")

In [ ]:
from collections import Counter
for item in Counter(refCounts).most_common():
    ref      = item[0]
    cnts     = item[1]
    discID   = arts.discogsUtils.getArtistID(ref)
    url      = arts.getArtistRef(ref)
    savename = arts.getArtistSavename(discID)
    if isFile(savename):
        continue
    try:
        arts.downloadArtistURL(url, savename)
    except:
        pass

In [ ]:
cnts   = [x.split()[0] for x in vals.split("\n")]
styles = [" ".join(x.split()[1:]) for x in vals.split("\n")]
d2000  = dict(zip(cnts, styles))
saveFile(idata=d2000, ifile="1980styles.p")

In [ ]:
ifile = "/Volumes/Music/Discog/collections/US-1960-Soundtrack-9.p"
from webUtils import getWebData, getHTML
from searchUtils import findPatternExt
verydebug=False
fdata  = getFile(ifile)
bsdata = getHTML(fdata)

albumsDir = disc.getAlbumsDir()

albs = albums(disc)


if True:
    collectionsDir  = col.getCollectionsDir()
    collectionFiles = findPatternExt(collectionsDir, pattern=".", ext=".p")
    print("Found {0} downloaded collection files".format(len(collectionFiles)))
    
    artsDowns = {}
    albsDowns = {}
    
    for i,ifile in enumerate(reversed(collectionFiles)):
        print("\n\n\n")
        print("Parsing {0}/{1}: {2}".format(i,len(collectionFiles), ifile))
        
        bsdata = getHTML(ifile)
        artistDB  = {}

        h4s = bsdata.findAll("h4")
        if verydebug:
            print("Found {0} h4 tags".format(len(h4s)))
        
        for ih4,h4 in enumerate(h4s):
            spans = h4.findAll("span")
            
            #### Get Artist
            ref   = None
            if len(spans) == 0:
                ref = h4.find("a")
            else:
                ref = spans[0].find("a")
                
            if ref is None:
                continue
                
            if artsDowns.get(ref) is True:
                continue
                
            try:
                artisthref = ref.attrs.get('href')
                artist     = ref.text.strip()
            except:
                print("Could not get artist/href from {0}".format(ref))
                continue
                
            
            #### Get Album
        
            try:
                ref        = h4.find("a", {"class": "search_result_title"})
                albumhref  = ref.attrs.get('href')
                album      = ref.text.strip()
            except:
                print("Could not get album/href from {0}".format(ref))
                continue

            
            #print("Artist: {0} ---> {1}".format(artist,artisthref))
            #print("Album:  {0} ---> {1}".format(album,albumhref))

            artistID  = arts.discogsUtils.getArtistID(artisthref)
            modValue  = arts.discogsUtils.getDiscIDHashMod(discID=artistID, modval=disc.getMaxModVal())
            
            
            ##### Download Artist Data
            url      = arts.getArtistRef(artisthref)
            savename = arts.getArtistSavename(artistID)
            if savename is not None:
                if not isFile(savename):
                    try:
                        arts.downloadArtistURL(url, savename)
                    except:
                        artsDowns[url] = True
                        continue
                #print('\t',url)
                #print('\t',savename)
                
        
            if modValue is None:
                artistModDir = setDir(albumsDir, "NAN")
                artistIDDir = setDir(artistModDir, "NAN")
            else:
                artistModDir = setDir(albumsDir, str(modValue))
                artistIDDir = setDir(artistModDir, artistID)


            ##### Download Album Data
            try:
                mediaID = int(getBasename(albumhref))
            except:
                raise ValueError("Not sure about {0}".format(h4))

            #print('\tAlbumDir:',artistIDDir)
            #print('\tMediaID: ',mediaID)

            if not isDir(artistIDDir):
                mkDir(artistIDDir)
            savename = setFile(artistIDDir, "{0}.p".format(mediaID))
            #print('\tSavename:',savename)
            if isFile(savename):
                continue
     
            #print('\t',savename)
            #print("")
            #continue

            baseURL = disc.discogURL
            url     = urllib.parse.urljoin(baseURL, quote(albumhref))
            
            
            if albsDowns.get(url) is True:
                continue
                
            try:
                albs.downloadAlbumURL(url, savename)
            except:
                albsDowns[url] = True
                continue

In [ ]:
#col.createCollectionsData()
#col.downloadCollections()
#col.parseCollections()
#col.mergeCollections(debug=True, force=True)
#col.createCollectionDBs(debug=True)

In [ ]:
disc.moveAlbumFilesToNewModValue(newModValue=100, oldModValue=500)

# Artist Section

In [ ]:
db = disc.getArtistNameToIDsData()

In [ ]:
#arts = artists(disc)
#art.findKnownArtists(debug=True)
#art.findUnknownArtists(minVal=200, debug=True)
#art.downloadUnknownArtists(debug=True)
#arts.parseArtistFiles()
#art.searchDiscogForArtist("$uicideboy$")
#arts.buildMetadata()

# Albums Metadata

In [ ]:
artistNameToID   = disc.getArtistNameToIDData()
artistNameToIDs  = disc.getArtistNameToIDsData()
artistIDToName   = disc.getArtistIDToNameData()        
artistRefToID    = disc.getArtistRefToIDData()
artistIDToRef    = disc.getArtistIDToRefData()
artistRefToName  = disc.getArtistRefToNameData()
artistNameToRef  = disc.getArtistNameToRefData()
artistNameToRefs = disc.getArtistNameToRefsData()

In [ ]:
albumRefToName  = disc.getAlbumRefToNameData()
albumNameToRef  = disc.getAlbumNameToRefData()

artistIDCoreAlbumIDs   = disc.getArtistIDCoreAlbumNames()
artistIDAlbumIDs       = disc.getArtistIDAlbumNames()



In [ ]:
from collections import Counter
core = ["Albums"]
albumCntr = Counter()


print("{0: <4}{1: <50}{2: <15}{3: <15}{4: <12}{5: <12}{6: <12}{7: <12}{8: <12}{9: <12}".format(
    "#","File","Core IDs-IDs","IDs-IDs","Name-IDs","Name-Refs","ID-Names","ID-Refs","Ref-IDs","Ref-Names"))


for i,dbfile in enumerate(dbfiles):
    db = getFile(dbfile)
    for artistID,artistData in db.items():
        artist      = artistData['Artist']
        artistName  = dutils.getArtistName(artist)
        artistRef   = artistData['URL']
        #print("{0: <20}{1: <50}{2}".format(artistID,artist,artistRef))
        
    print("{0: <4}{1: <50}{2: <15}{3: <15}{4: <12}{5: <12}{6: <12}{7: <12}{8: <12}{9: <12}".format(
        i,dbfile,
        len(artistIDCoreAlbumIDs),len(artistIDAlbumIDs),
        len(albumNameToID),len(albumNameToRef),len(albumIDToName),
        len(albumIDToRef),len(albumRefToID),len(albumRefToName)))



In [ ]:
ifile="/Volumes/Music/Discog/artists/0/9900.p"
fdata = getHTML(getFile(ifile, version=2))
an = artistName(fdata)
an.getArtist()
#name = artistName(fdata)
#name


In [ ]:
cnts   = [x.split()[0] for x in vals.split("\n")]
styles = [" ".join(x.split()[1:]) for x in vals.split("\n")]
d2000  = dict(zip(cnts, styles))
saveFile(idata=d2000, ifile="soundtracks.p")

# Albums

In [ ]:
albs = albums(disc)
albs.parseAlbumFiles()

In [ ]:
getFile('/Volumes/Music/Discog/albums-db/NAN-DB.p')

In [ ]:
import random
albs = albums(disc)
nErrs = False
while nErrs is False:
    nErrs = True
    albs.downloadAlbumDataData(int(random.uniform(0,disc.getMaxModVal()))) #, mediaTypes=None)        
    if False:
        nErrs = False
        print("Sleeping for 60 seconds...")
        sleep(60)
#art.findKnownArtists(debug=True)
#art.findUnknownArtists(minVal=200, debug=True)
#art.downloadUnknownArtists(debug=True)
#arts.parseArtistFiles()
#art.searchDiscogForArtist("$uicideboy$")
#arts.buildMetadata()

In [ ]:
%load_ext autoreload
%autoreload

from artist import artist
from artists import artists
art = artist()
arts = artists(disc)

toget=getFile(ifile="toget3.p")
done = {}
i = 0
errs = {}
n = len(toget)
for artistID,artistRef in toget.items():
    i += 1
    ifile = arts.getArtistSavename(artistID)
    print(i,'/',n,'\t',ifile)
    info  = art.getData(ifile)
    pages = info.pages
    
    if pages.redo is True:
        url      = arts.getArtistURL(artistRef, 1)
        savename = arts.getArtistSavename(artistID, 1)
        try:
            arts.downloadArtistURL(url=url, savename=savename, force=True, debug=True)            
        except:
            errs[artistID] = artistRef
            print("\tErr with {0}".format(artistRef))

In [ ]:
import sqlite3
db = sqlite3.connect("test.db")
cur = db.cursor()
cur.execute("create table people (first varchar, last varchar, age integer)")
cur.execute('insert into people values ("Steve", "Cassidy", 21)')
cur.execute('insert into people values ("Mary", "Mullins", 23)')
db.commit()